In [4]:
import mlflow
from mlflow.tracking import MlflowClient


mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.create_experiment(name="deploy-model-via-flask1",
#                         artifact_location="s3://ars-mlops-projects/mlflow/")

In [5]:
mlflow.set_experiment("deploy-model-via-flask1")



<Experiment: artifact_location='s3://ars-mlops-projects/mlflow/', experiment_id='2', lifecycle_stage='active', name='deploy-model-via-flask1', tags={}>

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import pickle

### Load and prepare dataframe

In [6]:
from sklearn import datasets
iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [7]:
iris_df['target']  = iris.target
iris_df.target.value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

In [8]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


### Segregate train and test dataset

In [9]:
x = iris_df.drop('target',axis=1)
y = iris_df['target']
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [11]:
with mlflow.start_run():
    mlflow.xgboost.autolog()
    from sklearn import metrics
    #mlflow.log_params({"developer" : "Arshad","ID":"XZZ7200","Dataset":"IRIS","random_state":"42","test_size":"0.33","algorithm":"XGBOOST"})
    xgb_classifer = XGBClassifier(objective='binary:logistic')
    xgb_classifer.fit(X_train,y_train)
    y_pred = xgb_classifer.predict(X_test)
    accuracy = metrics.accuracy_score(y_test,y_pred)
    metrics = {"Accuracy":accuracy}

In [10]:
%%bash
export GIT_PYTHON_REFRESH=quiet

In [13]:
pickle.dump(xgb_classifer,open('model','wb'))

In [14]:
model = pickle.load(open('model','rb'))

In [15]:
model.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 1, 2, 1, 2])

### Serve the model via Flask web app

In [3]:
%%writefile '/root/AWS Projects/AWS-Projects/ModelDeployment/Connect to model registry/predict.py'
#%%writefile '/home/ec2-user/notebooks/scripts/predict.py'
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import numpy as np
import mlflow
import boto3


from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://65.1.107.253:5000")
#RUN_ID = 'ebea9df24cd44f5a8617c8db2a4b09dd'
#logged_model = f'runs:/{RUN_ID}/model'
ssm = boto3.client('ssm',region_name='ap-south-1')
RUN_ID = ssm.get_parameter(Name='RUN_ID')['Parameter']['Value']
logged_model = f's3://ars-mlops-projects/mlflow/{RUN_ID}/artifacts/model'
loaded_model = mlflow.pyfunc.load_model(logged_model)
model = mlflow.pyfunc.load_model(logged_model)


def predictc(features):
    print("Inside predictc")
    predictions = model.predict(features)
    print(predictions)
    return predictions[0]
app = Flask('app')

@app.route('/predict',methods=['POST'])
def predict_endpoint():
    features = request.get_json()
    print("Inside predict_endpoint")
    print(features)
    feature_df = pd.DataFrame.from_dict([features])
    
    predictions = predictc(feature_df)
    print(predictions)
    
    result = {
        'Class' : str(predictions),
        'ModelVersion' : RUN_ID
    }
    print(result)
    
    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True,host='0.0.0.0',port=9697)

Writing /root/AWS Projects/AWS-Projects/ModelDeployment/Connect to model registry/predict.py


In [4]:
%%writefile '/root/AWS Projects/AWS-Projects/ModelDeployment/Connect to model registry/test_iris_model.py'
#%%writefile '/home/ec2-user/notebooks/scripts/test_iris_model.py'
import predict
import pandas as pd
import numpy as np
import requests

"""df  = pd.DataFrame.from_dict([{
    'sepal length (cm)': 5.1,
    'sepal width (cm)' : 3.5,
    'petal length (cm)': 1.4,
    'petal width (cm)' : 0.2
}])"""
features = {
    'sepal length (cm)': 5.1,
    'sepal width (cm)' : 3.5,
    'petal length (cm)': 1.4,
    'petal width (cm)' : 0.2
}

url = 'http://localhost:9697/predict'
print(features)
response = requests.post(url,json=features)
print(response.json)

Writing /root/AWS Projects/AWS-Projects/ModelDeployment/Connect to model registry/test_iris_model.py


In [10]:
import mlflow
RUN_ID = 'ebea9df24cd44f5a8617c8db2a4b09dd'
logged_model = f'runs:/{RUN_ID}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 1, 2, 1, 2])

In [12]:
MLFLOW_TRACKING_URI = 'http://65.1.107.253:5000'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [20]:
f'{path}/MLmodel'

'/tmp/tmpeeaodk6n/model/MLmodel'

In [6]:
import mlflow
model_name = 'iris_model'
model_version = 1
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

In [12]:
%%bash

aws ssm get-parameter --region "ap-south-1" --name "RUN_ID" 

{
    "Parameter": {
        "Name": "RUN_ID", 
        "DataType": "text", 
        "LastModifiedDate": 1658945300.442, 
        "Value": "ebea9df24cd44f5a8617c8db2a4b09dd", 
        "Version": 1, 
        "Type": "String", 
        "ARN": "arn:aws:ssm:ap-south-1:832173187970:parameter/RUN_ID"
    }
}


In [4]:
import boto3
ssm = boto3.client('ssm',region_name='ap-south-1')


In [16]:
ssm.get_parameter(Name='RUN_ID')['Parameter']['Value']

'ebea9df24cd44f5a8617c8db2a4b09dd'

In [2]:
pwd

'/root/AWS Projects/AWS-Projects/ModelDeployment/Connect to model registry'